# Modelowanie liczby roszczeń

### Generowanie niejednorodnego procesu Poissona metodą przerzedzania

In [2]:
import numpy as np
import pandas as pd

In [3]:
def nhpp(lambda_func,lambda_func_params, t_max):
    """
    Funkcja generuje niejednorodny proces Poissona metodą przerzerzedzania.

    Argumenty:
        lambda_func: funkcja
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności
        t_max: float
            Maksymalny czas symulacji
        lambda_max: float
            Ograniczenie górne funkcji lambda_func
    Zwraca:
        numpy.ndarray
            Tablica czasów.
    """

    S = 0
    N = 0
    S_tab = [0]
    lambda_max = lambda_func(0,lambda_func_params)[1]
    
    while S < t_max:
        U = np.random.uniform(0,1)
        S=S+(-1/lambda_max)*np.log(U)
        if S > t_max:
            break
        U = np.random.uniform(0,1)
        if U <= lambda_func(S,lambda_func_params)[0]/lambda_max:
            N = N + 1
            S_tab.append(S)

    return N, S_tab

In [4]:
def sin_int_fun(t,lambda_func_params):
    """
    Sinusoidalna funkcja intensywności.

    Argumenty:
        t: float
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności.
            W tym przypadku listę zawierającą: [lambda_0, lambda_1, T, phi]
    Zwraca:
        float
            Wartość funkcji intensywności.
        float
            Górne ograniczenie funkcji.
            W tym przypadku lambda_0+lambda_1.    
    """
    return lambda_func_params[0]+lambda_func_params[1]*np.sin(2*np.pi*t/lambda_func_params[2]+lambda_func_params[3]), lambda_func_params[0]+lambda_func_params[1]

In [5]:
def exp_int_fun(t,lambda_func_params):
    """
    Wykładnicza funkcja intensywności.

    Argumenty:
        t: float
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności. 
            W tym przypadku listę zawierającą: [lambda_0, lambda]
    Zwraca:
        float
            Wartość funkcji intensywności.
        float
            Górne ograniczenie funkcji.
            W tym przypadku lambda_0/lambda.
    """
    return lambda_func_params[0]*np.exp(-lambda_func_params[1]*t), lambda_func_params[0]*lambda_func_params[1]

In [6]:
nhpp(sin_int_fun,[1,3,1,0],10)

(18,
 [0,
  0.16800746679510425,
  0.20544825558608237,
  0.230752527714401,
  1.3343086723601536,
  2.3034215721093387,
  2.402974342613465,
  3.272349311347649,
  3.5115180379524706,
  4.516637603582349,
  5.148199761533021,
  5.167078926638778,
  5.169687846119377,
  6.179598262930948,
  7.287850367244566,
  7.345031397463598,
  9.230069099531548,
  9.230232667800243,
  9.381810579611127])

In [7]:
nhpp(exp_int_fun,[2,1/10],100)

(2, [0, 28.675494018880947, 29.806906719338194])

Empiryczny proces $N_t$.

In [8]:
data = pd.read_csv("https://github.com/ndzadz/mgr/blob/main/data_us_1990_2022.csv?raw=true",encoding="latin1",sep=";",header=0)
p95 = np.percentile(data[data["Insured Damages, Adjusted (\'000 US$)"].isna()==0]["Insured Damages, Adjusted (\'000 US$)"],95)
data = data[(data["Insured Damages, Adjusted (\'000 US$)"].isna()==0)&(data["Insured Damages, Adjusted (\'000 US$)"]<p95)]
data = data.rename(columns={'ï»¿Dis No':'Dis No'})

In [9]:
N = pd.concat([pd.Series([0]),pd.Series(data.groupby('Year').count()['Dis No'])]).reset_index()[0].rename('N')

In [10]:
N

0      0
1     17
2     28
3     29
4     25
5     11
6     18
7     12
8     26
9     26
10    16
11     7
12    10
13    13
14    13
15    13
16     9
17    12
18    22
19    13
20    10
21     9
22    16
23    20
24    20
25    15
26    25
27    21
28    32
29     4
30    10
31     8
32     1
33     2
Name: N, dtype: int64

Metoda najmniejszej sumy kwadratów

In [11]:
from scipy.optimize import minimize

Sinusoidalna funkcja intensywności

In [12]:
def sum_of_squares_sin(lambda_func_params):
    t = np.array(range(len(N)))
    return np.sum(np.array((N-sin_int_fun(t, lambda_func_params)[0])**2))

In [13]:
sum_of_squares_sin([1,3,1,0])

9015.000000000011

In [24]:
bnds = ((0, None), (0, None), (0, None), (0, None)) # ograniczenia na wartości parametrów
params = [1.0, 1.0, 1.0, 1.0] # Przyjmujemy początkowe wartości parametrów
result = minimize(sum_of_squares_sin, params, bounds=bnds, method='L-BFGS-B')
print(result)


      fun: 1575.267861512148
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.27373657e-05, -2.27373677e-05,  7.44648792e-02, -1.81898925e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 290
      nit: 44
     njev: 58
   status: 0
  success: True
        x: array([14.51045062,  6.56946102,  1.0560816 ,  9.69327362])


Symulacja procesu z sinusoidalną funkcją intensywności dla wyliczonych parametrów

In [15]:
n = nhpp(sin_int_fun,result.x,33)
print(n)

(481, [0, 0.0015968001456393953, 0.044540780689898724, 0.14857750170082348, 0.2259940399613202, 0.36145512473700214, 0.3974616274738798, 0.5111947813088492, 0.5753549910383158, 0.577091868557167, 0.6193240696456578, 0.6615226969668337, 0.7215162202214541, 0.7532499745775989, 0.7885298447941169, 0.826626812286541, 0.8499309378782457, 0.9090009823696233, 0.972455478506745, 1.014232308185542, 1.0569962513338216, 1.1598345098011342, 1.1818538647760088, 1.1945211808787506, 1.2382761343875606, 1.308712853932947, 1.3313787740912035, 1.35463995214077, 1.3985923965570985, 1.463380170252298, 1.5175495392843237, 1.5301359984877152, 1.6022351047614383, 1.6143341702824738, 1.6349890733701156, 1.7294496761759703, 1.7441106558705555, 1.8286728724519454, 1.8374516425783984, 1.899669268822037, 1.9731008156105383, 2.0610416968092653, 2.115114726842378, 2.236842316518583, 2.293349205745966, 2.4496719060401153, 2.5288084454647035, 2.6346694660568892, 2.660972231676768, 2.692830148802128, 2.713297047564959

In [16]:
unique, counts = np.unique(np.ceil(n[1]), return_counts=True)
N_t = pd.Series(counts)
N_t[0]=0
N_t

0      0
1     18
2     22
3     16
4     10
5     19
6     16
7     10
8     11
9     19
10    15
11    13
12    12
13    12
14    14
15    13
16    15
17    12
18    21
19    12
20    12
21    22
22    16
23    12
24    11
25    10
26    15
27     9
28    15
29    15
30    20
31    21
32    14
33     9
dtype: int64

Wykładnicza funkcja intensywności

In [17]:
def sum_of_squares_exp(lambda_func_params):
    t = np.array(range(len(N)))
    return np.sum(np.array((N-exp_int_fun(t, lambda_func_params)[0])**2))

In [18]:
sum_of_squares_exp([1,3])

10006.16344218761

In [25]:
bnds = ((0, None), (0, None)) # ograniczenia na wartości parametrów
params = [1.0, 1.0] # Przyjmujemy początkowe wartości parametrów
result = minimize(sum_of_squares_exp, params, bounds=bnds, method='L-BFGS-B')
print(result)


      fun: 2130.1683437993447
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.0001819,  0.0721684])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 51
      nit: 9
     njev: 17
   status: 0
  success: True
        x: array([1.85912816e+01, 1.31208572e-02])


Symulacja procesu z sinusoidalną funkcją intensywności dla wyliczonych parametrów

In [150]:
n = nhpp(exp_int_fun,result.x,33)
print(n)

(4, [0, 2.210059237127692, 3.0576332719452273, 29.990610443917966, 31.1412473300267])


In [151]:
unique, counts = np.unique(np.ceil(n[1]), return_counts=True)
N_t = pd.Series(counts)
N_t[0]=0
N_t

0    0
1    1
2    1
3    1
4    1
dtype: int64